In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [2]:
data = pd.read_csv("f:/datasets/CreditRisk.csv")

In [3]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
data = data.dropna()

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [6]:
categorical_feature = [feature for feature in data.columns if data[feature].dtypes == 'O']

In [7]:
for feature in categorical_feature :
    data[feature] = le.fit_transform(data[feature])

In [8]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,0,1,1,1.0,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,1,0.0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,2,1,1,0.0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,3,1,0,0.0,0,0,6000,0.0,141.0,360.0,1.0,2,1
5,4,1,1,2.0,0,1,5417,4196.0,267.0,360.0,1.0,2,1


In [9]:
X = data.iloc[:, 1:12]
y = data[['Loan_Status']]

## Keras Tuner

In [107]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2, 20)):
        model.add(layers.Dense(units = hp.Int('Units_' + str(i), min_value = 32, max_value = 512, step = 30), 
                               activation='relu'))
        model.add(layers.Dense(units = 1, activation = 'sigmoid'))
        model.compile(
            optimizer = keras.optimizers.Adam(
                hp.Choice('Learning_rate', [1e-2, 1e-3, 1e-4])), 
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])
    return model

In [108]:
tunner = RandomSearch(build_model, 
                      objective = 'val_accuracy',
                      max_trials = 5,
                      executions_per_trial = 3,
                      directory = 'f:\Demo_Project',
                      project_name = 'CreditRisk'
                     )

In [109]:
tunner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
Units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 30, 'sampling': None}
Learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
Units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 30, 'sampling': None}


In [110]:
from sklearn.model_selection import train_test_split

In [111]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 111)

In [112]:
tunner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 5 Complete [00h 00m 11s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7207792401313782
Total elapsed time: 00h 00m 44s
INFO:tensorflow:Oracle triggered exit


In [113]:
tunner.results_summary()

Results summary
Results in f:\Demo_Project\CreditRisk
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 3
Units_0: 512
Learning_rate: 0.01
Units_1: 92
Units_2: 32
Score: 0.7207792401313782
Trial summary
Hyperparameters:
num_layers: 4
Units_0: 152
Learning_rate: 0.001
Units_1: 182
Units_2: 392
Units_3: 32
Score: 0.7207792401313782
Trial summary
Hyperparameters:
num_layers: 4
Units_0: 92
Learning_rate: 0.01
Units_1: 452
Units_2: 242
Units_3: 92
Score: 0.7207792401313782
Trial summary
Hyperparameters:
num_layers: 3
Units_0: 182
Learning_rate: 0.001
Units_1: 242
Units_2: 482
Units_3: 512
Score: 0.7207792401313782
Trial summary
Hyperparameters:
num_layers: 10
Units_0: 452
Learning_rate: 0.0001
Units_1: 62
Units_2: 62
Units_3: 422
Units_4: 32
Units_5: 32
Units_6: 32
Units_7: 32
Units_8: 32
Units_9: 32
Score: 0.7207792401313782
